In [3]:
%load_ext autoreload
%autoreload 2

from plt_utils import *
from test_levels import *
import gym
print(gym.__version__)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.26.2


## policy computation discounted 
- policy_update corresponds to $f_v$ <br>
- value_update(iterations =1) corresponds to $L_{\text{policy}}$ <br>
- value_update_seidel(iterations =1) uses updated values of v 
as soon that they are updated like gauss-seidel method. <br>

- value_iteration is applying $L_{f_v}v = Uv$ (theorem 1.3.5)  <br>
- backward_induction is value_iteration with $v^{T+1} =0$ and time dependence (theorem 1.2.1) <br>
- policy_iteration is generalized_iteration because of (theorem 1.3.6) <br>
we don't apply $L_{\pi}$ infinite times but to convergence or $1000$ 
iterations.

the code is obvious, there is a small difference in the formula <br> 
because rewards are defined differently so you take it in the sum <br>
that way you get averaged reward corresponding to our definition <br>
(To make backward induction time dependent just make env.P[state][action] 
time dependent.)


In [4]:
def states(env): return range(env.observation_space.n)
def actions(env, state): return env.P[state].keys()
# infmetric, norm, only works for v
def oometric(v1, v2): return max(abs(v1[i]-v2[i]) for i in v1.keys())

def value_update(env, alpha, v, policy, iterations=1, eps=10**(-4)):
    for _ in range(iterations): 
        vold = v.copy() 
        for state in states(env): 
            v[state] = sum(pij*(r+alpha*vold[nextstate]) 
                           if not(done) else pij*r for (pij,nextstate,r,done) in env.P[state][policy[state]])
        if oometric(v, vold) < eps: break #convergence

def value_update_seidel(env, alpha, v, policy, iterations=1, eps=10**(-4)):
    for _ in range(iterations): 
        vold = v.copy() 
        for state in states(env): 
            v[state] = sum(pij*(r+alpha*v[nextstate]) # v instead vold, an OG error  for finite problem
                           if not(done) else pij*r for (pij,nextstate,r,done) in env.P[state][policy[state]])
        if oometric(v, vold) < eps: break #convergence


def policy_update(env, alpha, v, policy):
    for state in states(env):
        max_a, max_val = 0, -float("inf") #to select the argmax
        for action in actions(env, state):
            val = sum(pij*(r+alpha*v[nextstate]) if not(done) else pij*r for (pij,nextstate,r,done) in env.P[state][action])
            max_a, max_val = (action, val) if val > max_val else (max_a, max_val)
        policy[state] = max_a

def value_iteration(env,alpha,max_iter = 30,eps = 10**(-6)):
    v = {state:0 for state in states(env)} 
    policy = {state:0 for state in states(env)} 
    vv,pp = [],[] #these are for plotting

    for i in range(max_iter):
        policy_update(env,alpha,v,policy)
        value_update(env,alpha,v,policy,iterations=1)
        vv.append(v.copy()) # history of value functions
        pp.append(policy.copy()) # policy history
        if i>2 and oometric(vv[-1], vv[-2]) < eps: break
    return vv,pp

def backward_induction(env,alpha,T):
    return value_iteration(env,alpha,T,0)

def generalized_iteration(env,alpha,inner_iter=1,max_iter = 30,eps = 10**(-6)):
    v = {state:0 for state in states(env)} 
    policy = {state:0 for state in states(env)} 
    vv,pp = [],[] #these are for plotting

    for i in range(max_iter):
        value_update(env,alpha,v,policy,inner_iter,eps)
        policy_update(env,alpha,v,policy)
        vv.append(v.copy()) # history of value functions
        pp.append(policy.copy()) # policy history
        if i>2 and pp[-1]==pp[-2]==pp[-3]: break
    return vv,pp

def policy_iteration(env,alpha,max_iter = 30,eps = 10**(-6)):
    return generalized_iteration(env=env,alpha=alpha,inner_iter=10**3,max_iter = max_iter,eps = eps)

In [5]:
env = time_level()
# env = envrandom()
# env=gym.make('CliffWalking-v0')
# env= gym.make('Taxi-v3') # use value iteration
vv,pp = backward_induction(env,alpha=1,T=1000) #programming task 1
# vv,pp = policy_iteration(env,alpha=0.999,max_iter=50,eps=10**(-6)) #programming task 2
# vv,pp = value_iteration(env,alpha=0.999,max_iter=300,eps = 0.001) #programming task 3
sol = list(pp[-1].items()) # asked form of the policy in class
print(sol)
print(f"val(0) = {vv[-1][0]} at time 1 for finite horizon")
intvp(vv,pp)

[(0, 0), (1, 3), (2, 3), (3, 3), (4, 3), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 1), (16, 0), (17, 0), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 0), (25, 0), (26, 2), (27, 2), (28, 2), (29, 2), (30, 1), (31, 1), (32, 0), (33, 0), (34, 2), (35, 2), (36, 3), (37, 2), (38, 1), (39, 1), (40, 0), (41, 0), (42, 2), (43, 0), (44, 0), (45, 2), (46, 1), (47, 1), (48, 0), (49, 0), (50, 2), (51, 0), (52, 0), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 0), (60, 0), (61, 2), (62, 2), (63, 0)]
val(0) = 0.8334426747830409 at time 1 for finite horizon


interactive(children=(IntSlider(value=0, description='iterations', max=999), IntSlider(value=0, description='r…

## policy evaluation discounted
- value_eval is for evaluating a policy, basically it is a big  <br>
average of the rewards times the probability of getting it <br>
lot of terms can be reused, value_update does it basically  <br>
- MC_eval just is MC estimation of the total reward

In [6]:
def finite_value_eval(env,alpha,pp):
    v = {state:0 for state in states(env)} 
    vv= []
    for p in pp:
        value_update(env,alpha,v,p,1,0)
        vv.append(v.copy())
    print(f"finite val(0) = {vv[-1][0]}")
    return vv

def infinite_value_eval(env,alpha,policy,eps=10**(-6)):
    v = {state:0 for state in states(env)} 
    value_update(env,alpha,v,policy,10**5,eps)
    print(f"infinite val(0) = {v[0]}")
    return v 

# does MC estimation of the expected reward
def MC_eval(env,pol,alpha,T=10**3,nsim = 10**3):
    running_sum = 0
    for _ in range(nsim):
        state = env.reset()[0]
        total_reward = 0
        for t in range(T): 
            action = pol[t][state]
            state, reward, done, _, _ = env.step(action)
            total_reward += alpha**t*reward
            if done: break
        env.close()
        running_sum +=total_reward/nsim
    print(f"infinite MC val(0) = {running_sum}")
    return running_sum

def random_policy(env,T):
    return [{state:env.action_space.sample() 
            for state in states(env)}
            for _ in range(T)]

def infinite_pol_rep(p,T):
    return {t: {i:p[i] 
              for i in range(env.observation_space.n)}
                for t in range(T)}

def finite_pol_rep(pp,T):
    return {t: {i:pp[-t-1][i] 
              for i in range(env.observation_space.n)}
                for t in range(T)}

In [7]:
env = time_level()
alpha = 0.999
T = 10**3
pvv,ppp = value_iteration(env,alpha=alpha,max_iter=600,eps=10**(-6)) 
infinite_value_eval(env,alpha,ppp[-1])
MC_eval(env,infinite_pol_rep(ppp[-1],T),alpha=alpha,T=T,nsim=10**3)
print(f"reference = {pvv[-1][0]}")

infinite val(0) = 0.7076089300881696
infinite MC val(0) = 0.7162973732271237
reference = 0.7074435915028664


In [8]:
env = time_level()
alpha=0.999
T = 30
bvv,bpp = backward_induction(env,alpha=alpha,T=T) 
finite_value_eval(env,alpha=alpha,pp=bpp)
MC_eval(env,finite_pol_rep(bpp,T),alpha=alpha,T=T,nsim=10**3)
print(f"reference = {bvv[-1][0]}")

finite val(0) = 0.011042662185476242
infinite MC val(0) = 0.00974619105238034
reference = 0.011042662185476242


In [9]:
# env = trivial()
env = time_level()
alpha=0.999
T = 400
rpp = random_policy(env,T)
finite_value_eval(env,alpha,rpp)
MC_eval(env,finite_pol_rep(rpp,T),alpha=alpha,T=T,nsim=10**3)
print()

finite val(0) = 9.163854374067952e-05
infinite MC val(0) = 0.0009531108968798942



## policy evaluation average
We use theorem 1.4.14 to calculate $P^*$ as 
$$
P^* = \lim_{n \to \infty} (0.5 P + 0.5I)^{n} .
$$ 

In [13]:
from scipy.sparse import dok_array,eye
from scipy.sparse.linalg import norm 

def P_from_policy(env,policy):
    n = len(states(env))
    P = dok_array((n, n), dtype=np.float32)
    for i in states(env):
        for (pij,j,_,_) in env.P[i][policy[i]]:
            P[i,j] += pij 
    return P

def power_convergence(P,max_iter,eps =10**(-12)):
    Q=P
    for _ in range(max_iter):
        Qnew = Q@Q
        if norm(Q-Qnew)<eps:break
        Q = Qnew
    return Q 

def treshold(A,eps = 10**(-10)):
    A.data[abs(A.data) < eps] = 0
    A.eliminate_zeros()
    return A

def stationary_matrix(P, eps = 10**(-10)):
    Pstar = power_convergence(0.9*P+0.1*eye(P.shape[0]),15,eps)
    return treshold(Pstar,eps) 

def infinite_value_eval_average(env,policy,rewd):
    P = P_from_policy(env,policy)
    r = np.array([rewd[state][policy[state]] for state in states(env)])
    Q = stationary_matrix(P) 
    phi = Q@r
    return {state:phi[state] for state in states(env)}

In [18]:
env = time_level()
_,ppp = policy_iteration(env,alpha=1) 
pvv = [infinite_value_eval(env,1,ppp[-1])]
P = P_from_policy(env,ppp[-1])
Pstar = stationary_matrix(P)
print(f"ind0: P, ind1: Pstar, ind2: Zinv")
intPP([P]+[Pstar]+[eye(Pstar.shape[0])-P+Pstar])

infinite val(0) = 0.8333661771717522
ind0: P, ind1: Pstar, ind2: Zinv


interactive(children=(IntSlider(value=0, description='index', max=2), Output()), _dom_classes=('widget-interac…

If the reward vector is $1$ at the goal, the average reward must correspond to 
the discounted reward with $\alpha =1$ which in this case can be calculated with 
the discounted tools. Here we check that they are similar.

In [19]:
rewd = {state:{action:0 if state != 63 else 1 for action in actions(env,state)} for state in states(env)} 
average_pvv = infinite_value_eval_average(env,ppp[-1],rewd) 
df ={state:average_pvv[state]-pvv[-1][state] for state in states(env)}
print("difference between average_pvv and pvv[-1]")
intvp([df],[ppp[-1]])

difference between average_pvv and pvv[-1]


interactive(children=(IntSlider(value=0, description='iterations', max=0), IntSlider(value=0, description='row…

## policy computation average
This is Algorithm 1.4.2

In [61]:
from scipy.sparse.linalg import bicgstab

def better_average_actions(env,rewd,phi,u0,policy,eps = 10**(-4)):
    sol = {state:[] for state in states(env)}
    done = True
    for state in states(env): 
        for action in actions(env,state):
            if policy[state] == action: continue
            phiaction = sum(pij*phi[nextstate] for (pij,nextstate,_,_) in env.P[state][action])
            if phiaction> phi[state]+eps:
                sol[state].append(action)
                done = False
                continue
            if abs(phiaction - phi[state])<eps:
                LHS14 = rewd[state][action]+sum(pij*u0[nextstate] for (pij,nextstate,_,_) in env.P[state][action])
                if LHS14-phi[state]-u0[state]>eps:
                    sol[state].append(action)
                    done =False
    return sol,done

def policy_update_average(env,policy,B):
    for state in states(env):
        if len(B[state])!=0: 
            if policy[state] !=B[state][0]: 
                policy[state] = B[state][0]
            elif len(B[state])>=2:
                policy[state]= B[state][1]



def policy_iteration_average(env,rewd,max_iter = 30):
    policy = {state:0 for state in states(env)} 
    vv,pp,u0 = [],[],np.zeros(env.observation_space.n)
    I = eye(env.observation_space.n)
    for i in range(max_iter):
        r = np.array([rewd[state][policy[state]] for state in states(env)])
        P = P_from_policy(env,policy)
        Pstar = stationary_matrix(P)
        Zinv = I-P+Pstar
        phi =  Pstar@r 
        u0, _ = bicgstab(Zinv,r-Zinv@phi,x0=u0)
        B,done = better_average_actions(env,rewd,phi,u0,policy)
        vv.append({state:phi[state] for state in states(env)})
        pp.append(policy.copy())
        if done or pp[-1] in pp[-3:-1]: break
        policy_update_average(env,policy,B)
    return vv,pp

In [66]:
env = time_level()
# env = envrandom()
rewd = {state:{action:0 if state != 63 else 1 for action in actions(env,state)} for state in states(env)} 
pvv, ppp = policy_iteration(env,alpha=1)
apvv, appp = policy_iteration_average(env,rewd,max_iter=30)
intvp(apvv,appp)

interactive(children=(IntSlider(value=0, description='iterations', max=11), IntSlider(value=0, description='ro…

In [67]:
random_average_vv = infinite_value_eval_average(env,random_policy(env,1)[0],rewd) # programming task 4.3
df1 ={state:apvv[-1][state]-pvv[-1][state] for state in states(env)} 
df2 ={state:apvv[-1][state]-random_average_vv[state] for state in states(env)} 
print("it0: apvv[-1] vs pvv[-1], it1: apvv[-1] vs random policy")
print("the comparison is the difference")
intvp([df1,df2],2*[appp[-1]]) # programming task 4.4

it0: apvv[-1] vs pvv[-1], it1: apvv[-1] vs random policy
the comparison is the difference


interactive(children=(IntSlider(value=0, description='iterations', max=1), IntSlider(value=0, description='row…

## $\varepsilon$-greedy Q-learning
We assume we know the states and the actions ...,
we start of from a value function because otherwise 
sparse rewards are a problem 

In [14]:
from random import randint
def get_action_from_q(q,state):
    i = randint(0,3)
    max_a , max_val = i, q[state][i]
    for action,val in q[state].items():
        max_a, max_val = (action, val) if val > max_val else (max_a, max_val)
    return max_a,max_val

def get_q_from_v(env,v,alpha):
    return {state:{action:
      sum(pij*(r+alpha*v[nextstate]) if not(done) else pij*r
          for (pij,nextstate,r,done) in env.P[state][action]) for action in actions(env,state) } for state in states(env)}

def get_policy_from_q(env,q): # Programming task 5 (B)
    return {state:get_action_from_q(q,state)[0] for state in states(env)}

def val_est(state):
    row,col = divmod(state,8)
    return (row+col)/14

def q_0(env):
    return {state:{action:0 for action in actions(env,state) } for state in states(env)}

def q_est(env):
    return {state:{action:val_est(state) for action in actions(env,state) } for state in states(env)}

def Q_learning(env,v,alpha,gamma,T=10**3,nsim = 10**3):
    goal_count = 0
    q = get_q_from_v(env,v,alpha) # uses P
    # q = q_est(env)
    # q = q_0(env)
    for i in range(nsim):
        state = env.reset()[0] 
        for t in range(T):
            action, _ = get_action_from_q(q,state) if randint(0,30)!=0 else (randint(0,3),1)
            state, reward, done, _, _ = env.step(action)
            _, qval2 = get_action_from_q(q,state)
            q[state][action] = (1-gamma)*(q[state][action])+ gamma*(reward+alpha*qval2)
            goal_count +=reward
            if done: break
        env.close()
    print(f"reached goal {goal_count} times in sims")
    return q

In [17]:
# env = time_level()
env = envrandom()
alpha = 1
rp = random_policy(env,1)[0]
rv = infinite_value_eval(env,alpha,rp) #uses P
q= Q_learning(env,rv,alpha=alpha,gamma=0.2,T=2*10**2,nsim=10**4) # Programming task 5
qp = get_policy_from_q(env,q)

# Programming task 5 (C) and (D)
qv = infinite_value_eval(env,alpha,qp)
dv = {state:(qv[state]-rv[state]) for state in states(env)}
# intvp([rv], [rp])
print("it2: improvement of the value")
intvp([qv,dv], 2*[qp])
# visq(q)

infinite val(0) = 0.0
reached goal 434.0 times in sims
infinite val(0) = 0.10272103289778609
it2: improvement of the value


interactive(children=(IntSlider(value=0, description='iterations', max=1), IntSlider(value=0, description='row…